In [1]:
import pandas as pd
import math

In [2]:
df = pd.read_csv("NCLab_df.csv")

In [3]:
# Initialize the dictionary with each key as "INPUT|CONTROLLER|x_a" where x_a is activation type, and value being a list of 
# [num_mentions, pos_or_neg]

interaction_dict = {}
i = 0
for i in range(0, len(df.INPUT)):
    if df.NUM_LABEL[i] == 1:
        interaction_dict[f"{df.INPUT[i]}|{df.CONTROLLER[i]}|pos"] = [df.COUNTER[i], df.NUM_LABEL[i]]
    elif df.NUM_LABEL[i] == -1:
        interaction_dict[f"{df.OUTPUT[i]}|{df.CONTROLLER[i]}|neg"] = [df.COUNTER[i], df.NUM_LABEL[i]]
    elif df.NUM_LABEL[i] == 0:
        interaction_dict[f"{df.OUTPUT[i]}|{df.CONTROLLER[i]}|nan"] = [df.COUNTER[i], df.NUM_LABEL[i]]
    i += 1

In [4]:
# We make a proportion dictionary with each key as "Input/Output|Controller" and each value as [pos_count, total_count]

# initialize the dictionary
prop_dict = {}

for key in interaction_dict:
    prop_dict[key[:-4]] = [0, 0]


In [5]:
# Add just Input/Output|Controller as key to prop_dict, and if the activation is postive, add to the first element in the list value. Otherwise
# just add to total.

for key in interaction_dict:
    split_key = key.split("|")
    count = interaction_dict[key][0]
    if split_key[2] == "pos":
        prop_dict[key[:-4]][0] += count
        prop_dict[key[:-4]][1] += count

    if split_key[2] == "neg":
        prop_dict[key[:-4]][1] += count
        
    if split_key[2] == "nan":
        prop_dict[key[:-4]][0] += count/2
        prop_dict[key[:-4]][1] += count

In [6]:
# for each entry in prop_dict, divide pos by total to get the proportion. Then input into the modified tanh function to get our edge value.
# subtract 1 for tanh, leave -1 out for sigmoid

for key in prop_dict:
    proportion = prop_dict[key][0] / prop_dict[key][1]
    y = (1/(1+math.exp(-5*(proportion-.5))))
    prop_dict[key].append(y)

In [7]:
# Change dictionary's form into dataframe-compatible form.
df_dict = {
    "INPUT": [],
    "CONTROLLER": [],
    "EDGE": [],
    "POS": [],
    "TOTAL": [],
}
for key in prop_dict:
    met_input, met_cont = key.split("|")
    df_dict["INPUT"].append(met_input)
    df_dict["CONTROLLER"].append(met_cont)
    df_dict["EDGE"].append(prop_dict[key][2])
    df_dict["POS"].append(prop_dict[key][0])
    df_dict["TOTAL"].append(prop_dict[key][1])


In [8]:
prop_df = pd.DataFrame.from_dict(df_dict)

In [9]:
prop_df

,INPUT,CONTROLLER,EDGE,POS,TOTAL
0,Bax::uniprot:Q07812.t,Bcl-2::uniprot:P10415.t,0.075858,0.0,5
1,E5,Bax::uniprot:Q07812.t,0.924142,4.0,4
2,Jagged1::uniprot:P78504.t,Bax::uniprot:Q07812.t,0.075858,0.0,4
3,Notch1::uniprot:P46531.t,Bax::uniprot:Q07812.t,0.075858,0.0,4
4,PCNA::uniprot:P12004.t,Bax::uniprot:Q07812.t,0.075858,0.0,4
...,...,...,...,...,...
2995,E3,Eq::uaz:UAZ4571,0.924142,1.0,1
2996,E3,Fpr2::uniprot:P25090,0.924142,1.0,1
2997,E3,Gls1::uniprot:O94925,0.924142,1.0,1
2998,Δ::uaz:UAZCE94.m,SMTs::uaz:UAZ534D5473,0.500000,0.5,1


In [10]:
prop_df.to_csv("NC_tanh.csv")